In [1]:
from tokenizers import Tokenizer
from bastionlab.polars.policy import Policy, Aggregation, Log
from bastionlab.polars import train_test_split
import polars as pl
from bastionlab import Connection
from bastionlab.tokenizers import RemoteTokenizer


file_path = "./SMSSpamCollection"

# Read CSV file using Polars and rename columns with `text`, `label`
df = pl.read_csv(file_path, has_header=False, sep="\t", new_columns=["label", "text"])

# Transform `spam` labels to `1` and `0` for any other column label
df = df.with_column(
    pl.when(pl.col("label") == "spam").then(1).otherwise(0).alias("label")
)

# View the first few elements of the DataFrame
df.head()

/home/kbamponsem/base/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


label,text
i64,str
0,"""Go until juron..."
0,"""Ok lar... Joki..."
1,"""Free entry in ..."
0,"""U dun say so e..."
0,"""Nah I don't th..."


In [2]:
tokenizer = RemoteTokenizer.from_hugging_face_pretrained("distilbert-base-uncased")
tokenizer.enable_padding(length=32)
tokenizer.enable_truncation(max_length=32)

connection = Connection("localhost")

policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())

# df = pl.DataFrame(
#     {
#         "ids": [[2]],
#         "mask": [[1]],
#     }
# )
rdf = connection.client.polars.send_df(df, policy=policy)

rdf.column("label").to_tensor()
# input_ids, attention_mask = tokenizer.encode(rs)
# print("\n---\n", input_ids.collect())

RemoteTensor(identifier=96f49364-3dda-4c1f-a5ca-9097bca3c3b4)

RemoteTensor(identifier=1fc91469-ff6b-4198-9851-de099c54531a)

{"DataFrameScan":{"df":{"columns":[{"name":"ids","datatype":"List","values":[{"name":"","datatype":"Float64","values":[]}]},{"name":"mask","datatype":"List","values":[{"name":"","datatype":"Float64","values":[]}]}]},"schema":{"inner":{"ids":{"List":"Float64"},"mask":{"List":"Float64"}}},"output_schema":null,"projection":null,"selection":null}}
